<a href="https://colab.research.google.com/github/palexey2006/Ml-Engineering/blob/main/RNN_models/Simple_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
from tqdm import tqdm
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


input_size = 28
sequence_length = 28
num_classes = 10
learning_rate = 0.001
hidden_size = 256
num_layers = 2
batch_size = 64
num_epochs = 7

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        out = self.fc(out)
        return out

train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


model = RNN(input_size, hidden_size,num_layers, num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch + 1} / {num_epochs} Loss: {loss.item(): .3f}")



def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)


            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

100%|██████████| 938/938 [00:08<00:00, 106.19it/s]


Epoch: 1 / 7 Loss:  0.183


100%|██████████| 938/938 [00:09<00:00, 96.25it/s] 


Epoch: 2 / 7 Loss:  0.051


100%|██████████| 938/938 [00:09<00:00, 97.82it/s] 


...

Epoch: 5 / 7 Loss:  0.392


100%|██████████| 938/938 [00:09<00:00, 101.66it/s]


Epoch: 6 / 7 Loss:  0.169


100%|██████████| 938/938 [00:09<00:00, 100.01it/s]


Epoch: 7 / 7 Loss:  0.372
Accuracy on training set: 97.97
Accuracy on test set: 97.35
